# Segment 4 Lab 1

## Introduction to this segment's labs

# Introducing: The Price Is Right

We are going to build a multi-agent framework that will:
1. Pull RSS feeds from the web and interpret them, looking for promising deals (multi-shot prompting)
2. Use RAG and a Frontier Model to estimate the price of a product, using a knowledge base
3. Use a QLoRA fine tuned LLM to estimate the price
4. Send a push notification when it finds a deal that's worth significantly less than it estimates

### We'll be going through this fast - the idea is for you to come back and try it yourself!


## First: Let's look at the Agent code for interpreting RSS feeds using multi-shot prompting

In [ ]:
# Some imports

from dotenv import load_dotenv
from agents.deals import ScrapedDeal
from openai import OpenAI
import logging

In [ ]:
# Set up our env variables as before

load_dotenv()
openai = OpenAI()
MODEL = "gpt-4o-mini"

In [ ]:
# I wrote this code to fetch RSS feeds

scraped_deals = ScrapedDeal.fetch(show_progress=True)

In [ ]:
print(scraped_deals[3].describe())

In [ ]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [ ]:
USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
"""

USER_PROMPT_SUFFIX = """\n\nStrictly respond in JSON and include exactly 5 deals, no more.
Your response must follow this format:

{"deals": [
    {
        "product_description": "Your summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    {
        "product_description": "Your summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 210.30,
        "url": "the url as provided"
    },
    ...
]}
"""

In [ ]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [ ]:
user_prompt = make_user_prompt(scraped_deals)
print(user_prompt[:2000])

In [ ]:
# Create the user prompt that reflects the deals we have scraped

user_prompt = make_user_prompt(scraped_deals)

# Call OpenAI using their Python client. Request a response in JSON and stream it back.

stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
  ],
    max_tokens=1000,
    stream=True,
    response_format={"type": "json_object"}
)

response = ""
for chunk in stream:
    chunk_text = chunk.choices[0].delta.content or ''
    response += chunk_text
    print(chunk_text, end="", flush=True)

In [ ]:
import json
json.loads(response)

# Putting this into an "Agent"

I've packaged this code into a class called `ScannerAgent`

There are various Agent frameworks that add an abstraction layer, but in our case it's easy just to write the code directly.

In [ ]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
from agents.scanner_agent import ScannerAgent

scanner = ScannerAgent()
print(scanner.scan())